# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [21]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

Save the IAM role in a variable called `role`. This would be useful when training the model.

In [22]:
role = sagemaker.get_execution_role()
print(role)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


arn:aws:iam::789453636658:role/service-role/AmazonSageMaker-ExecutionRole-20230817T141326


In [23]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://object-detection-project-1/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [4]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [24]:
# build and push the docker image. This code can be commented after being ran once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
#!sh ./docker/build_and_push.sh $image_name

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [25]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

789453636658.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20230818012136


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [27]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/faster_restnet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/faster_restnet50.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint

mkdir: cannot create directory ‘/tmp/checkpoint’: File exists
mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2023-08-20 01:19:26--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.31.128, 2607:f8b0:4004:c1b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.31.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘/tmp/faster_restnet50.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 11.2M 18s
    50K .......... .......... .......... .......... ..........  0% 23.8M 13s
   100K .......... .......... .......... .......... ..........  0% 10.5M 15s
   150K .......... .......... .......... .......... ..........  0% 80.2M 12s
   200K .......... .......... .......... .......... ..........  0% 17.4M 12s
   250

  5000K .......... .......... .......... .......... ..........  2% 53.0M 5s
  5050K .......... .......... .......... .......... ..........  2% 44.8M 5s
  5100K .......... .......... .......... .......... ..........  2%  252M 5s
  5150K .......... .......... .......... .......... ..........  2% 40.7M 5s
  5200K .......... .......... .......... .......... ..........  2% 44.8M 5s
  5250K .......... .......... .......... .......... ..........  2% 43.8M 5s
  5300K .......... .......... .......... .......... ..........  2% 64.8M 5s
  5350K .......... .......... .......... .......... ..........  2% 82.5M 5s
  5400K .......... .......... .......... .......... ..........  2% 44.6M 5s
  5450K .......... .......... .......... .......... ..........  2% 48.6M 5s
  5500K .......... .......... .......... .......... ..........  2% 39.7M 5s
  5550K .......... .......... .......... .......... ..........  2%  363M 5s
  5600K .......... .......... .......... .......... ..........  2% 51.5M 5s
  5650K ....

 10400K .......... .......... .......... .......... ..........  5% 41.0M 4s
 10450K .......... .......... .......... .......... ..........  5%  340M 4s
 10500K .......... .......... .......... .......... ..........  5% 50.9M 4s
 10550K .......... .......... .......... .......... ..........  5% 41.9M 4s
 10600K .......... .......... .......... .......... ..........  5% 42.4M 4s
 10650K .......... .......... .......... .......... ..........  5% 41.9M 4s
 10700K .......... .......... .......... .......... ..........  5%  337M 4s
 10750K .......... .......... .......... .......... ..........  5% 47.0M 4s
 10800K .......... .......... .......... .......... ..........  5% 53.4M 4s
 10850K .......... .......... .......... .......... ..........  5% 43.6M 4s
 10900K .......... .......... .......... .......... ..........  5% 58.3M 4s
 10950K .......... .......... .......... .......... ..........  5%  115M 4s
 11000K .......... .......... .......... .......... ..........  5% 40.1M 4s
 11050K ....

 15800K .......... .......... .......... .......... ..........  7%  361M 3s
 15850K .......... .......... .......... .......... ..........  7%  264M 3s
 15900K .......... .......... .......... .......... ..........  7%  185M 3s
 15950K .......... .......... .......... .......... ..........  7%  335M 3s
 16000K .......... .......... .......... .......... ..........  7%  307M 3s
 16050K .......... .......... .......... .......... ..........  7%  356M 3s
 16100K .......... .......... .......... .......... ..........  7%  289M 3s
 16150K .......... .......... .......... .......... ..........  7%  358M 3s
 16200K .......... .......... .......... .......... ..........  7%  256M 3s
 16250K .......... .......... .......... .......... ..........  7%  309M 3s
 16300K .......... .......... .......... .......... ..........  7%  286M 3s
 16350K .......... .......... .......... .......... ..........  7% 1.30M 4s
 16400K .......... .......... .......... .......... ..........  7%  154M 4s
 16450K ....

 21200K .......... .......... .......... .......... .......... 10%  308M 3s
 21250K .......... .......... .......... .......... .......... 10%  360M 3s
 21300K .......... .......... .......... .......... .......... 10%  322M 3s
 21350K .......... .......... .......... .......... .......... 10%  368M 3s
 21400K .......... .......... .......... .......... .......... 10%  104M 3s
 21450K .......... .......... .......... .......... .......... 10%  337M 3s
 21500K .......... .......... .......... .......... .......... 10%  329M 3s
 21550K .......... .......... .......... .......... .......... 10%  362M 3s
 21600K .......... .......... .......... .......... .......... 10% 7.75M 3s
 21650K .......... .......... .......... .......... .......... 10%  347M 3s
 21700K .......... .......... .......... .......... .......... 10% 52.5M 3s
 21750K .......... .......... .......... .......... .......... 10%  340M 3s
 21800K .......... .......... .......... .......... .......... 10%  283M 3s
 21850K ....

...

188600K .......... .......... .......... .......... .......... 91%  128M 0s
188650K .......... .......... .......... .......... .......... 91% 92.4M 0s
188700K .......... .......... .......... .......... .......... 91%  157M 0s
188750K .......... .......... .......... .......... .......... 91% 68.1M 0s
188800K .......... .......... .......... .......... .......... 91% 68.5M 0s
188850K .......... .......... .......... .......... .......... 91%  152M 0s
188900K .......... .......... .......... .......... .......... 91%  117M 0s
188950K .......... .......... .......... .......... .......... 91% 96.5M 0s
189000K .......... .......... .......... .......... .......... 91%  100M 0s
189050K .......... .......... .......... .......... .......... 91%  129M 0s
189100K .......... .......... .......... .......... .......... 91% 16.5M 0s
189150K .......... .......... .......... .......... .......... 91% 95.6M 0s
189200K .......... .......... .......... .......... .......... 91% 71.9M 0s
189250K ....

194000K .......... .......... .......... .......... .......... 93%  267M 0s
194050K .......... .......... .......... .......... .......... 93%  366M 0s
194100K .......... .......... .......... .......... .......... 93%  371M 0s
194150K .......... .......... .......... .......... .......... 93%  370M 0s
194200K .......... .......... .......... .......... .......... 93%  289M 0s
194250K .......... .......... .......... .......... .......... 93%  375M 0s
194300K .......... .......... .......... .......... .......... 93%  366M 0s
194350K .......... .......... .......... .......... .......... 93%  311M 0s
194400K .......... .......... .......... .......... .......... 93%  322M 0s
194450K .......... .......... .......... .......... .......... 93%  357M 0s
194500K .......... .......... .......... .......... .......... 93%  323M 0s
194550K .......... .......... .......... .......... .......... 93%  370M 0s
194600K .......... .......... .......... .......... .......... 94%  162M 0s
194650K ....

199400K .......... .......... .......... .......... .......... 96%  171M 0s
199450K .......... .......... .......... .......... .......... 96%  135M 0s
199500K .......... .......... .......... .......... .......... 96%  376M 0s
199550K .......... .......... .......... .......... .......... 96%  380M 0s
199600K .......... .......... .......... .......... .......... 96%  332M 0s
199650K .......... .......... .......... .......... .......... 96%  177M 0s
199700K .......... .......... .......... .......... .......... 96%  147M 0s
199750K .......... .......... .......... .......... .......... 96%  283M 0s
199800K .......... .......... .......... .......... .......... 96%  222M 0s
199850K .......... .......... .......... .......... .......... 96%  156M 0s
199900K .......... .......... .......... .......... .......... 96%  149M 0s
199950K .......... .......... .......... .......... .......... 96%  283M 0s
200000K .......... .......... .......... .......... .......... 96% 95.7M 0s
200050K ....

204800K .......... .......... .......... .......... .......... 98%  296M 0s
204850K .......... .......... .......... .......... .......... 98%  348M 0s
204900K .......... .......... .......... .......... .......... 98%  334M 0s
204950K .......... .......... .......... .......... .......... 99%  382M 0s
205000K .......... .......... .......... .......... .......... 99%  320M 0s
205050K .......... .......... .......... .......... .......... 99%  342M 0s
205100K .......... .......... .......... .......... .......... 99%  378M 0s
205150K .......... .......... .......... .......... .......... 99%  380M 0s
205200K .......... .......... .......... .......... .......... 99%  288M 0s
205250K .......... .......... .......... .......... .......... 99%  379M 0s
205300K .......... .......... .......... .......... .......... 99%  311M 0s
205350K .......... .......... .......... .......... .......... 99%  380M 0s
205400K .......... .......... .......... .......... .......... 99%  327M 0s
205450K ....

faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [33]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.trn1.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: tf2-object-detection-2023-08-20-02-40-07-675


2023-08-20 02:40:18 Starting - Starting the training job...
2023-08-20 02:40:33 Starting - Preparing the instances for training......
2023-08-20 02:41:25 Downloading - Downloading input data...
2023-08-20 02:41:50 Training - Downloading the training image......
2023-08-20 02:43:11 Training - Training image download completed. Training in progress....2023-08-20 02:43:37,677 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-08-20 02:43:37,679 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-08-20 02:43:37,691 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-08-20 02:43:37,693 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-08-20 02:43:37,704 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-08-20 02:43:37,706 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0820 02:43:48.090100 140458478384960 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use `tf.cast` instead.
W0820 02:43:51.263307 140458478384960 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
/usr/local/lib/python3.8/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/Fa

INFO:tensorflow:Step 600 per-step time 5.685s
I0820 03:42:27.484549 140458478384960 model_lib_v2.py:705] Step 600 per-step time 5.685s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.12119347,
 'Loss/BoxClassifierLoss/localization_loss': 0.24820256,
 'Loss/RPNLoss/localization_loss': 0.36003608,
 'Loss/RPNLoss/objectness_loss': 0.0392582,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 0.7686903,
 'learning_rate': 0.019960001}
I0820 03:42:27.484817 140458478384960 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.12119347,
 'Loss/BoxClassifierLoss/localization_loss': 0.24820256,
 'Loss/RPNLoss/localization_loss': 0.36003608,
 'Loss/RPNLoss/objectness_loss': 0.0392582,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 0.7686903,
 'learning_rate': 0.019960001}
INFO:tensorflow:Step 700 per-step time 5.689s
I0820 03:51:56.377803 140458478384960 model_lib_v2.py:705] Step 700 per-step time 5.689s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classifi

INFO:tensorflow:Step 1600 per-step time 5.691s
I0820 05:17:20.288712 140458478384960 model_lib_v2.py:705] Step 1600 per-step time 5.691s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.22634453,
 'Loss/BoxClassifierLoss/localization_loss': 0.5184082,
 'Loss/RPNLoss/localization_loss': 0.5145136,
 'Loss/RPNLoss/objectness_loss': 0.06753098,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.3267974,
 'learning_rate': 0.05156}
I0820 05:17:20.288997 140458478384960 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.22634453,
 'Loss/BoxClassifierLoss/localization_loss': 0.5184082,
 'Loss/RPNLoss/localization_loss': 0.5145136,
 'Loss/RPNLoss/objectness_loss': 0.06753098,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.3267974,
 'learning_rate': 0.05156}
INFO:tensorflow:Step 1700 per-step time 5.694s
I0820 05:26:49.719836 140458478384960 model_lib_v2.py:705] Step 1700 per-step time 5.694s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification

/usr/local/lib/python3.8/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
INFO:tensorflow:depth of additional conv before box predictor: 0
I0820 05:55:36.812602 140183622838080 convolutional_keras_box_predictor.py:152] depth of additional conv before box predictor: 0
Instructions for updating:
Use ref() instead.
W0820 05:55:42.409377 140183622838080 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:460: Tensor.experimental_ref (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use ref() instead.
Instructions for updating:
Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.
See `

INFO:tensorflow:depth of additional conv before box predictor: 0
I0820 05:56:00.682927 139688022734656 convolutional_keras_box_predictor.py:152] depth of additional conv before box predictor: 0
Instructions for updating:
Use ref() instead.
W0820 05:56:06.463510 139688022734656 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:460: Tensor.experimental_ref (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use ref() instead.
I0820 05:56:11.226894 139688022734656 signature_serialization.py:148] Function `call_func` contains input name(s) resource with unsupported characters which will be renamed to mask_rcnn_keras_box_predictor_mask_rcnn_class_head_classpredictor_dense_biasadd_readvariableop_resource in the SavedModel.
W0820 05:56:14.035677 139688022734656 save_impl.py:66] Skipping full serialization of Keras layer <object_detection.meta_architectures.faster_rcnn_met

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)
